In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import numpy as np
import os
from scipy import spatial
from scipy import ndimage
from scipy.signal import find_peaks
from mpl_toolkits.mplot3d import Axes3D
from skimage import feature
from skimage import morphology
import pickle
from scipy import stats
from scipy import signal
import time
import imp
import h5py
#import fl
import warnings
import sys
#from termcolor import colored
#from multiprocessing import Pool
from nd2reader import ND2Reader
#from scipy.misc import imread
import statsmodels.api as sm
from pathlib import Path
from scipy.stats import sem
from scipy.spatial.transform import Rotation as Rot
import itertools
import math as m

#Exp parameters

#HEATMAPS Density calculations
inj_radius=37.5
z_led_dist=4
vol_zled_inin=z_led_dist*m.pi*inj_radius**2
n_obs_zled=2
stained_fr=0.025
corr_n_plts=1/((n_obs_zled)*(stained_fr))
corr_zled=corr_n_plts/vol_zled_inin

#Define exp series here!!!!
#---------------------------------------------------

exp_series_names=['PAR4 cohort','Inhibitor cohort','Thrombin cohort']
#exp_series=exp_series_names
#

# Change some settings 
#----------------------------------------------------------------------
%matplotlib inline
warnings.simplefilter(action = "ignore", category = FutureWarning)

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
#pd.option_context('display.max_rows', None, 'display.max_columns', None)


#AESTHETICS
#Parameters for image export 
#---------------------------------------------------------------------------
#matplotlib.rcParams['pdf.fonttype'] = 42
#matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['image.cmap'] = 'jet'
pal_bW='Greys'


#Palettes
#---------------------------------------------------------------------------
pal_bw='Greys'
pal_few_c='Set1'
pal_many_c='muted'#pal_many_c='Paired'


#Plot styles 
#---------------------------------------------------------------------------
#sns.set_style('darkgrid')
#sns.set_style('whitegrid')
sns.set_style('ticks')
#sns.set_context("talk") #paper,notebook,talk,poster
sns.set_context("talk")
plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['image.interpolation'] = 'none'


#Plotting parameters
plot_height=6
plot_aspect=1.25
ci_1=70
time_lim=[0,600]
time_ticks=[0,250,500]
title_pos=0.9
title_size=25

tgroup_names=np.arange(0,578,6)
#---------------------------------------------------------------------------
# Dictionary with categories of columns: 
dic_df_cols={'pos_num':['x_s', 'ys', 'zs','zz', 'zled','zf','dist_c', 'dist_cz','zled','zf','depth'],
             'pos_cat':['position','inside_injury', 'height', 'z_pos'],
             'time':['frame','time','minute',],
             'mov':['dvx', 'dvy', 'dvz', 'dv','stab','mov_class', 'movement','cont', 'cont_p','dvz_s','cont_s'],
             'fluo':['c0_mean', 'c0_max','c1_mean', 'c1_max','c2_mean', 'c2_max'],
             'shape':['elong', 'flatness','eigval_0', 'eigval_1', 'eigval_2'],
             "neighbours":['nb_i_0','nb_i_1','nb_i_2','nb_d_0','nb_d_1','nb_d_2','nba_d_5','nba_d_10','nba_d_15'],
             "clusters":['cl_idx_5','cl_idx_7.5','cl_idx_10','cl_idx_15','cl_idx_20','cld'],
             "tracking":['tracked','nrtracks','tracknr','cont_tot','displ_tot','dvz_tot',],#'dvy_tot'
             'info':['path','inj', 'exp', 'date','mouse','cohort', 'treatment'] 
           }
#---------------------------------------------------------------------------
# Orders of variables for plotting 
#Create lists that specify the order of the variable values in the graphs
bol_order=[True,False]
mov_class_order=['still','contractile','loose','none']
mov_class_order1=['still','contractile','loose']
movement_order=['immobile','contracting','drifting','unstable','none']
movement_order1=['immobile','contracting','drifting','unstable']
movements_order2=['immobile','contracting','drifting']
position_order=['head','tail','outside']
height_order=['bottom','middle','top']

#Inhibitor orders for graphs 
#-------------------------------------------------------
#inh_order1=['ctrl','biva','par4--','cang','par4--biva']
#inh_order=['ctrl','saline','biva','par4--','cang','par4--biva']
#inh_order1=['saline','biva','par4--','cang','par4--biva']
#inh_order_thrombin=['saline','biva','par4--','par4--biva']

#inh_order2=['saline','dmso','biva','sq','cang']
#Inhibitor labels for graphs 
#-------------------------------------------------------
#inh_labels=['Control','Saline','Bivalirudin','PAR4-/-','Cangrelor','PAR4-/- + Bivalirudin']
#

#---------------------------------------------------------------------------

def params_dict(x_var,kind):
    params=dict(x="time",hue=x_var,ci=70, kind=kind, height=8,aspect=1.25)
    return params

#---------------------------------------------------------------------------
#Kwarg dictionaries for plotting
def create_params():
    global params_line,params_line1,params_point,params_heatmap
    
    params_line=dict(x="time",hue='inh',ci=70, kind="line", hue_order=inh_order,
                     height=4,aspect=1.25)

    params_line1=dict(x="time",hue='inh',ci=95, kind="line", hue_order=inh_order,
                      height=6,aspect=1.25)

    params_point=dict(x="minute",hue='inh',ci=70, kind="point", hue_order=inh_order,
                      height=6,aspect=1.25)# hue_labels=inh_labels,
    params_heatmap=dict(xticklabels=15, cmap="turbo",vmax=max_col,vmin=min_col)



#pal = dict(zip(days, sns.color_palette("Set1", n_colors=len(days))))

# Leftover figure aesthetics
#----------------------------------------------------------

#g objects:
#g.set_titles('Thrombus platelet count as function of time, per treatment')
#g.set(xlim=[0,600],xticks=[0,250,500], yticks=[0,10000,20000])
#g.despine(lower=True)
#g.fig.set_size_inches(15,6)
#g.set_titles('Thrombus platelet count as function of time, per treatment')


# Pyplot commands
#plt.legend(bbox_to_anchor=(0.7, 1), loc=2, borderaxespad=0.,labels=inh_labels1)
#plt.title('Thrombus platelet count as function of time, per treatment')
#plt.savefig('plt_count_alltreatments.pdf',transparent=True)
#plt.savefig('plt_count_alltreatments.svg',transparent=True)
#plt.show()


#imp.reload(fl)
%matplotlib inline
warnings.simplefilter(action = "ignore", category = FutureWarning)